# QAOA TSP

In [4]:
import networkx as nx
from matplotlib import pyplot as plt
#graph
class Graph:
    def __init__(self, edges_set):
        self.edges_set = edges_set
        self.node_set = []
        for i in edges_set:
            if (i.start_node not in self.node_set):
                self.node_set.append(i.start_node)
            if (i.end_node not in self.node_set):
                self.node_set.append(i.end_node)

class Edge:
    def __init__(self, start_node, end_node):
        self.start_node = start_node
        self.end_node = end_node

set_edges = [Edge(0, 1), Edge(1, 2), Edge(2, 3), Edge(3, 0), Edge(2, 0), Edge(1, 3), Edge(0, 4), Edge(1, 4), Edge(2, 4), Edge(3, 4), Edge(4, 5), Edge(5, 0), Edge(2, 6), Edge(6, 3)]

G = nx.Graph()

for z in set_edges:
    G.add_edge(str(z.start_node), str(z.end_node))

nx.draw(G)
plt.savefig('graph.png')
plt.clf()

<Figure size 432x288 with 0 Axes>

# Initialization

In [5]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np
import math
from matplotlib import pyplot as plt
import random
from scipy.optimize import minimize

# Defines the list of qubits

num = 5
depth = 4
rep = 1000
qubits = QuantumRegister(num)
circuit = QuantumCircuit(qubits)
#distances and coupling might be wrong
distances = [15, 5, 3, 5, 15, 4, 3, 4 , 15]
coupling = [(5, 5), (3, 3), (4, 4)]

# Defines the initialization
def initialization(qubits):
    max = int(math.sqrt(num))
    for i in range(0, max):
        circuit.x(qubits[i * max])
        circuit.x(qubits[i * max + 1])

        #SWAP qubit 0 and 2
        circuit.cx(qubits[i * max], qubits[i * max + 2])
        circuit.csx(qubits[i * max + 2], qubits[i * max])
        circuit.cx(qubits[i * max], qubits[i * max + 2])

        #SWAP qubit 1 and 2
        circuit.cx(qubits[i * max + 1], qubits[i * max + 2])
        circuit.csx(qubits[i * max + 2], qubits[i * max + 1])
        circuit.cx(qubits[i * max + 1], qubits[i * max + 2])

        circuit.x(qubits[i * max + 2])
        circuit.s(qubits[i * max + 2])
        circuit.z(qubits[i * max + 2])
        circuit.x(qubits[i * max + 2])

        circuit.x(qubits[i * max])
        circuit.t(qubits[i * max])
        circuit.x(qubits[i * max])

# Cost Unitary

In [6]:
# Defines the cost unitary

def cost_unitary(qubits, gamma):

    # Penalizes the distances

    for i in range (0, len(qubits)):
        circuit.rz(gamma * distances[i] / (2 * math.pi), qubits[i])
    
    # Penalizes the hard-constraint

    for i in coupling:
        circuit.rzz(5 * gamma / math.pi, qubits[coupling[0], coupling[1]])

# Mixer Unitary

In [7]:
# Defines the mixer unitary

def mixer_unitary(qubits, alpha):
    
    for i in range(0, math.sqrt(num)):
        circuit.rxx(-0.5*alpha/math.pi, qubits[i*math.sqrt(num)], qubits[i*math.sqrt(num)+1])
        circuit.rxx(-0.5*alpha/math.pi, qubits[i*math.sqrt(num) + 1], qubits[i*math.sqrt(num)+2])
        circuit.ryy(-0.5*alpha/math.pi, qubits[i*math.sqrt(num)], qubits[i*math.sqrt(num)+1])
        circuit.ryy(-0.5*alpha/math.pi, qubits[i*math.sqrt(num)+1], qubits[i*math.sqrt(num)+2])

# Create Circuit (except it doesn't work lol)

In [8]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter


def create_circuit(params):

    gamma = [params[0], params[2], params[4], params[6]]
    alpha = [params[1], params[3], params[5], params[7]]


    initialization(qubits)
    
    for i in range(0, depth):
        cost_unitary(qubits, gamma[i])
        mixer_unitary(qubits, alpha[i])
    measured = ClassicalRegister(num)
    circuit.append(measured)
    circuit.measure(qubits, measured)
    #circuit.append(cirq.measure(*qubits, key='x'))
    #print(circuit)

    simulator = Aer.get_backend('aer_simulator')
    simulation = execute(circuit, simulator, shots=1000)
    result = simulation.result()
    counts = result.get_counts(circuit)
    counts = str(counts)[2:].split(", ")
    new_res = []
    for i in range(0, rep):
        hold = []
        for j in range(0, num):
            hold.append(int(counts[j][i]))
        new_res.append(hold)

    return new_res

# Cost Function

In [9]:
# Defines the cost function

def cost_function(params):

    av = create_circuit(params)
    total_cost = 0
    for i in range(0, len(av)):
        for j in range(0, len(av[i])):
            total_cost += distances[j]*av[i][j]
        for j in coupling:
            total_cost += -5*(1 - 2*av[i][j[0]])*(1 - 2*av[i][j[1]]) 
    total_cost = float(total_cost)/rep

    print("Cost: "+str(total_cost))

    return total_cost

# Optimization Method (doesn't work yet either)

In [10]:
# Defines the optimization method

init =[float(random.randint(-314, 314))/float(100) for i in range(0, 8)]
out = minimize(cost_function, x0=init, method="COBYLA", options={'maxiter':100})
print(out)

optimal_params = out['x']
f = create_circuit(optimal_params)

# Creates visualization of the optimal state

nums = []
freq = []

for i in range(0, len(f)):
    number = 0
    for j in range(0, len(f[i])):
        number += 2**(len(f[i])-j-1)*f[i][j]
    if (number in nums):
        freq[nums.index(number)] = freq[nums.index(number)] + 1
    else:
        nums.append(number)
        freq.append(1)

freq = [s/sum(freq) for s in freq]

print(nums)
print(freq)

x = range(0, 2**num)
y = []
for i in range(0, len(x)):
    if (i in nums):
        y.append(freq[nums.index(i)])
    else:
        y.append(0)

plt.bar(x, y)
plt.show()

CircuitError: 'expected integer or slice index into register'